# 7 Days of Code - Alura

In [27]:
#importing data
import pandas as pd
from datetime import datetime

url = 'https://www.senado.gov.br/transparencia/LAI/verba/despesa_ceaps_'
ceaps = pd.concat(pd.read_csv(url+str(year)+'.csv', encoding='ISO-8859-1', sep=';', skiprows=1, decimal=',') for year in range(2019,2023))

In [16]:
#informações gerais
ceaps.info()
ceaps.describe()
print('# de senadores únicos: ',len(ceaps['SENADOR'].unique()))
print('# de fornecedores únicos: ',len(ceaps['FORNECEDOR'].unique()))
# fonte da solução: https://stackoverflow.com/questions/59236780/how-to-check-for-wrong-datetime-entries-python-pandas
ceaps['DATA_CORRETA'] = pd.to_datetime(ceaps['DATA'], format="%d/%m/%Y", errors='coerce')
date_error_list = ceaps.loc[ceaps['DATA_CORRETA'].isnull()]['DATA'].unique().tolist()
date_error_list

['31/01/0219', '05/05/0202', '29/10/0202']

Apenas 3 entradas erradas, simples de resolver. Se fossem mais casos, eu retiraria as linhas.

In [22]:

correção = {'31/01/0219':'31/01/2019', '05/05/0202': '05/05/2020', '29/10/0202': '29/10/2020'}
ceaps['DATA_CORRETA'] = ceaps['DATA_CORRETA'].fillna(pd.to_datetime(ceaps['DATA'].map(correção)))
ceaps.sort_values('DATA_CORRETA').head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,DATA_CORRETA
3060,2022,2,ROGÉRIO CARVALHO,"Passagens aéreas, aquáticas e terrestres nacio...",34.848.898/0001-74,Peakalika Turismo,SRPSJX,02/02/2002,"Companhia Aérea: LATAM, Localizador: SRPSJX. P...",2260.60,2174993,2002-02-02
2715,2022,2,PAULO ROCHA,"Aluguel de imóveis para escritório político, c...",02.558.157/0001-62,Telefônica do Brasil S.A.,1129495338,11/02/2002,NaN,1106.13,2174871,2002-02-11
208,2021,2,ALESSANDRO VIEIRA,"Passagens aéreas, aquáticas e terrestres nacio...",39.827.859/0001-04,MM TOUR,LNXEJH,19/02/2002,"Companhia Aérea: LATAM, Localizador: LNXEJH. P...",2666.43,2156738,2002-02-19
12347,2020,3,TASSO JEREISSATI,"Contratação de consultorias, assessorias, pesq...",20.647.510/0001-62,FAAV CONSULTORIA ECONOMICA E EMPRESARIAL LTDA,00000061,14/03/2002,NaN,14100.00,2143467,2002-03-14
2740,2022,3,PAULO ROCHA,"Passagens aéreas, aquáticas e terrestres nacio...",16.978.175/0001-08,Adria Viagens e Turismo Ltda,GENXJE,16/03/2002,"Companhia Aérea: LATAM, Localizador: GENXJE. P...",3553.23,2176808,2002-03-16


Como podemos observar os erros nas datas são mais profundos do que aparentavam. Para solucionar estes casos, vou extrair o dia da coluna "data correta" e formar uma nova coluna de data a partir das colunas de mês e ano originais do dataset.

In [26]:
ceaps['DIA'] = ceaps['DATA_CORRETA'].dt.day
ceaps['DATA'] = pd.to_datetime(dict(year=ceaps.ANO, month=ceaps.MES, day=ceaps.DIA))

ValueError: cannot assemble the datetimes: day is out of range for month